# Required packages

In [ ]:
# !pip install similaritymeasures
# !pip install scikit-learn-extra
# !pip install geopy

# Download the data

In [ ]:
# Step_1: Download data From: https://www.kaggle.com/datasets/vineethakkinapalli/citibike-bike-sharingnewyork-cityjan-to-apr-2021
# Step_2: Merge all csv files into one file
# from glob import glob
# combined_df = pd.DataFrame()
# for file in sorted(glob('CitiBike_Trip_Data/*/*.csv')):
#     df = pd.read_csv(file)
#     print(df.shape)
#     combined_df = pd.concat([combined_df, df])

# combined_df.to_csv('Citibike BikeSharing-NewYork-2020-2021-combined.csv', index=False)

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from Bike_utils_similarity.Exp import *
from Bike_utils_similarity.extra_utils import data_partition,write_
from Bike_utils_similarity.utils.pkl_utils import *
from Bike_utils_similarity.utils.time_utils import get_TimeStamp_str
from Bike_utils_similarity.utils.distance_extraction import get_distance
from Bike_utils_similarity.similarity_utils import similarity, get_hours_curve_data

import pandas as pd
from sklearn_extra.cluster import KMedoids
from sklearn.preprocessing import LabelEncoder

# Renaming cols to be similar to "2015-2016 dataset"

In [14]:
current_cols = ['tripduration', 'starttime', 'stoptime', 'start station id',
                'start station name', 'start station latitude',
                'start station longitude', 'end station id', 'end station name',
                'end station latitude', 'end station longitude', 'bikeid', 'usertype',
                'birth year', 'gender']
new_cols = [ 'Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID', 
                'Start Station Name', 'Start Station Latitude',
                'Start Station Longitude', 'End Station ID', 'End Station Name',
                'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
                'Birth Year', 'Gender']
cols_naming = {}
for i,j in zip(current_cols,new_cols):cols_naming[i]=j

df.rename(columns=cols_naming, inplace=True)

df['Trip_Duration_in_min'] = df['Trip Duration']/60
df['Trip_Duration_in_min'] = df['Trip_Duration_in_min'].astype(int)

# df.columns

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender', 'Trip_Duration_in_min'],
      dtype='object')

# Outlier Removal

In [ ]:
# ==========    Outlier Removal Start  =========================================================================================================================
# --------------- 1 start 
reduction_method = '-reduced_Extra-60'
df = df[df['Trip_Duration_in_min']<60]
# ==========    Outlier Removal End  =========================================================================================================================

# Data preprocessing

In [16]:
# --------------- 1 start
le = LabelEncoder()
df['User Type'] = le.fit_transform(df['User Type'])  

a = pd.to_datetime(df['Start Time'])
df['DO_week'] = a.dt.day_of_week
df['DO_month'] = a.dt.day
df['month'] = a.dt.month
df['hour'] = a.dt.hour
df['min'] = a.dt.minute
df['year'] = a.dt.year

df['age'] = df['year'] - df['Birth Year']

df['Start Hour'] = a.dt.hour 
# --------------- 1 end   

## Encoding "Start" & "End" Station_names

In [17]:
# ========== mappping_dic ===============================
# --------------- 2 start 
e = LabelEncoder()
df['Start Station Encoding'] = e.fit_transform(df['Start Station Name'])    
dd = df[['Start Station Name', 'Start Station Encoding']]
mappping_dic = dd.set_index('Start Station Name').T.to_dict('list')
# dump(mappping_dic,'mappping_dic')

for k,v in mappping_dic.items():
    mappping_dic[k] = v[0]
    
def custom_encoding(x):
    return mappping_dic[x] if x in mappping_dic.keys() else -1

df['End Station Encoding'] = df['End Station Name'].apply(lambda x:custom_encoding(x))
df = df[~(df['End Station Encoding']==-1) ]
dump(mappping_dic,'dump/mappping_dic-2020-2021'+reduction_method)
# --------------- 2 End

C:\Users\Fathalla\AppData\Local\Temp\ipykernel_27908\1276063541.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  mappping_dic = dd.set_index('Start Station Name').T.to_dict('list')


In [18]:
# --------------- 3 Start: Calc Distance
from Bike_utils_similarity.utils.distance_extraction import get_distance
df['distance'] = df[['Start Station Latitude',
                     'Start Station Longitude',
                     'End Station Latitude',
                     'End Station Longitude']].apply(lambda x: get_distance(x), axis=1)
# --------------- 3 End

# Save Modified DataFrame

In [19]:
df.to_csv('Citibike BikeSharing-NewYork-2020-2021-modified%s.csv'%reduction_method, index=False)

# Steps for the proposed method

In [20]:
# --------------- 4 start    "dic.csv"
count = 0
l=[]
for i in range(len(mappping_dic)):
    for j in range(i, len(mappping_dic)):
        l.append(['%d-%d'%(i,j), count])
        count+=1
dic_ = pd.DataFrame( np.array(l), columns = ['key', 'value'])     
dic_.to_csv('dump/dic-2020-2021%s.csv'%reduction_method, index=False)
# --------------- 4 End

In [21]:
# --------------- 5 start: Count number of trips between two stations
data_partition_ = data_partition(df.copy())

v_lst = list(mappping_dic.values())
z = np.zeros((len(v_lst), len(v_lst)), dtype=int)
mappping_dic = load_pkl('dump/mappping_dic-2020-2021') # =======================================

for i in range(len(mappping_dic.values())):
    for j in range(i, len(mappping_dic.values())):
        z[i,j] = data_partition_.get_data( v_lst[i], v_lst[j], info_='shape')
        
d = pd.DataFrame(z, columns=v_lst)
d.to_csv('dump/Trip count_martix-2020-2021%s.csv'%reduction_method, index=False)      
# --------------- 5 End    

In [22]:
# --------------- 6 start - Generate Curves data
Conf_matrix_cell_curve_data = {}
v_lst = list(mappping_dic.values())
data_partition_ = data_partition(df.copy())

for i in range(len(mappping_dic.values())):
    for j in range(i, len(mappping_dic.values())):
        dframe = data_partition_.get_data( v_lst[i], v_lst[j], info_='data')
        Conf_matrix_cell_curve_data['%s-%s'%(i,j)] = get_hours_curve_data(dframe)
        
dump(Conf_matrix_cell_curve_data,'dump/Conf_matrix_cell_curve_data-2020-2021'+reduction_method)    
# --------------- 6 End  

In [23]:
# --------------- 7 start - "dist_matrix"
Conf_matrix_cell_curve_data = load_pkl('dump/Conf_matrix_cell_curve_data-2020-2021'+reduction_method)
k_lst = list(Conf_matrix_cell_curve_data.keys())
dist_matrix = np.zeros((len(k_lst),len(k_lst)))

similarity_ = similarity(Conf_matrix_cell_curve_data)

for i in range(len(k_lst)):
    for j in range(i+1, len(k_lst)):
        x = similarity_.calc_similarity(k_lst[i], k_lst[j]) 
        dist_matrix[i,j] = x
        dist_matrix[j,i] = x
dist_matrix = pd.DataFrame( np.array(dist_matrix) )
dist_matrix.to_csv('dump/dist_matrix-2020-2021%s.csv'%reduction_method, index=False)
# --------------- 7 End

In [ ]:
# --------------- 8 "clustering.pkl"  KMedoids:
dist_matrix = pd.read_csv('dump/dist_matrix-2020-2021%s.csv'%reduction_method,index_col=None)
res={}

for i in range(2,11):
    clustering_algo = KMedoids(n_clusters=i, random_state=0)
    model = clustering_algo.fit(dist_matrix)
    res[str(i)]=model.labels_

dump(res, 'dump/clustering-%s-2020-2021%s'%(str(clustering_algo).split('(')[0], reduction_method)) #    of length 1326
print('dump/clustering-%s-2020-2021%s'%(str(clustering_algo).split('(')[0], reduction_method))
# --------------- 8 End

# Run Experiment

## Run all

In [27]:
clustering_method = 'KMedoids'

df = pd.read_csv('Citibike BikeSharing-NewYork-2020-2021-modified%s.csv'%reduction_method)
dic_df = pd.read_csv('dump/dic-2020-2021%s.csv'%reduction_method)
res = load_pkl('dump/clustering-'+clustering_method+'-2020-2021'+reduction_method)
data_partition_ = data_partition(df.copy())
cluster_df = None

for i in range(2,11):
    exp_df = None
    results = []
    tmp = dic_df.copy()
    tmp['cluster'] = res[str(i)]
    cluster_id_results = []
    for cluster_id in set(res[str(i)]):
        cluster_df = tmp[tmp['cluster']==cluster_id]
        cluster_id_df = pd.DataFrame()
        for k in cluster_df['key'].values: #
            w = [int(w) for w in k.split('-')]
            sub_df = data_partition_.get_data(w[0], w[1], info_='data')
            cluster_id_df = pd.concat([cluster_id_df, sub_df])
        # train and predict the data ===========================
        write_('output','\tcluster_df.shape ---------- cluster_id:%d'%cluster_id , 
                                                                      cluster_df.shape, 
                                                                      "  DF length:", 
                                                                      cluster_id_df.shape[0] )
        exp_res = ML_exp(cluster_id_df,
                         start_model=0,
                         end_model=10, # ==========================
                         cluster='C-%s:%d-k:%d'%(clustering_method[:6],i,cluster_id),
                         target=-1)
        # get the results =======================================
        cluster_id_results.extend(exp_res)

    all_res = pd.DataFrame(np.array(cluster_id_results), columns=results_cols)
    all_res.to_csv('res/cluster-%s-2020-2021 %d - %s%s.csv'%(clustering_method, i,
                                                            get_TimeStamp_str(), 
                                                            reduction_method), 
                                                            index=False)

	cluster_df.shape ---------- cluster_id:0 (1092, 3)   DF length: 298786 
	cluster_df.shape ---------- cluster_id:1 (286, 3)   DF length: 94324 


# ALL Data

In [ ]:
df = pd.read_csv('Citibike BikeSharing-NewYork-2020-2021-modified%s.csv'%reduction_method)
exp_res = ML_exp(df,
                 start_model=0,
                 end_model=10,
                 cluster='all_data',
                 target=-1)
all_res = pd.DataFrame(np.array(exp_res), columns=results_cols)

main_df = 'res/cluster-2020-2021 all_data - %s%s.csv'%(get_TimeStamp_str(),
                                                            reduction_method)
all_res.to_csv(main_df, index=False)    

# Summarize the results

In [ ]:
from Bike_utils_similarity.extra_utils import *
from Bike_utils_similarity.utils.time_utils import get_TimeStamp_str

import warnings;warnings.filterwarnings('ignore')
from glob import glob

# clustering_method = ''
files = sorted(glob('res/cluster-%s*.csv'%clustering_method)) 

rs = ['Trip_Duration_in_min', 'distance']
ms = ['RandomFore', 'BaggingReg', '<catboost.']

res_df = None
main_df = pd.read_csv(main_df)
for f in files:
    d_file = pd.read_csv(f)
    for r in rs:
        for m in ms:
            d = main_df.copy()
            d = d[( (d['model']==m) & (d['response']==r)) ]
            df = d_file.copy()
            df = df[( (df['model']==m) & (df['response']==r)) ]
            a = df.describe()
            a = a.loc[['mean'], :]  # 'mean', 'std'

            res_df = pd.concat([res_df, d,empty_row(d.columns), df,a, empty_row(d.columns, 4)])

res_df.to_excel('summary-%s-%s-%s.xlsx'%(clustering_method, get_TimeStamp_str(), reduction_method), index=False)